# Installing Dependencies

In [ ]:
# pip -q install --upgrade ruptures scikit-learn tqdm matplotlib joblib

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at e:\swe\python3.11.5\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [87]:
import os
import zipfile
import shutil
from datetime import date, datetime, timedelta
import datetime as dt
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.stats import kurtosis, skew
from scipy.signal import find_peaks

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import joblib
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

import ruptures as rpt

from tqdm import tqdm
tqdm.pandas()

# Data Loading

### Setup Folders for Data Processing

In [88]:
dataset_path = 'AI/DataSet/'
features_path = 'E4/'
label_path = 'Subj_metric'
processed_dataset_path = 'AI/Processed_Dataset/'
model_path = 'AI/Models/'

In [89]:
os.makedirs(processed_dataset_path, exist_ok=True)
os.makedirs(model_path, exist_ok=True)

### Process Features Folders

In [90]:
def process_features_folders(base_directory, processed_dataset_path):
    
    # Loop through serial numbers from 1 to 13
    for serial_number in tqdm(range(1, 14), desc="Processing folders"):
        folder_name = f'{serial_number}-E4-Drv{serial_number}'
        folder_path = os.path.join(base_directory, folder_name)

        # Check if the folder exists
        if os.path.exists(folder_path) and os.path.isdir(folder_path):
            
            # List files in the folder
            files_in_folder = os.listdir(folder_path)

            # Unzip each zip file in the folder
            for file_name in tqdm(files_in_folder, desc="Unzipping files", leave=False):
                file_path = os.path.join(folder_path, file_name)
                if file_name.endswith('.zip'):
                    
                    # Create a new folder for each unzipped content
                    new_folder_name = f'{serial_number}_unzipped'
                    new_folder_path = os.path.join(processed_dataset_path, new_folder_name)
                    os.makedirs(new_folder_path, exist_ok=True)

                    with zipfile.ZipFile(file_path, 'r') as zip_ref:
                        zip_ref.extractall(new_folder_path)

        else:
            tqdm.write(f"Folder {folder_name} not found.")

In [91]:
process_features_folders((dataset_path + features_path), processed_dataset_path)

Processing folders: 100%|██████████| 13/13 [00:01<00:00,  7.32it/s]


### Process Label Folder

In [92]:
def process_label_files(base_directory, processed_dataset_path):
    
    # Loop through serial numbers from 1 to 13
    for serial_number in tqdm(range(1, 14), desc="Processing files"):
        file_name = f'SM_Drv{serial_number}.csv'
        file_path = os.path.join(base_directory, file_name)

        # Check if the file exists
        if os.path.exists(file_path) and os.path.isfile(file_path):

            # New filepath to copy file to
            new_folder_name = f'{serial_number}_unzipped'
            new_folder_path = os.path.join(processed_dataset_path, new_folder_name)

            # Copy the file to the new folder
            new_file_path = os.path.join(new_folder_path, file_name)
            shutil.copy(file_path, new_file_path)

        else:
            tqdm.write(f"File {file_path} not found.")

In [93]:
process_label_files((dataset_path + label_path), processed_dataset_path)

Processing files: 100%|██████████| 13/13 [00:00<00:00, 393.55it/s]


### Load from Processed Folder

#### HR Data Processing

- HR data relatively less than EDA and TEMP so data is repeated 4 times to have similar amount of data as the other readings

In [94]:
def process_hr_data(base_path):

    # Loop through folders with names in the format X_unzipped
    for serial_number in tqdm(range(1, 14), desc="Processing HR Data"):
        folder_name = f"{serial_number}_unzipped"
        current_folder_path = os.path.join(base_path, folder_name)

        # Check if the folder exists
        if os.path.exists(current_folder_path) and os.path.isdir(current_folder_path):
            # Define the path to the HR.csv file in the current folder
            hr_data_path = os.path.join(current_folder_path, "HR.csv")

            try:
                # Load, repeat, and save the HR data to HR_new.csv
                hr_data = np.loadtxt(hr_data_path, delimiter=',')
                hr_data = np.repeat(hr_data, 4)
                np.savetxt(os.path.join(current_folder_path, "HR_new.csv"), hr_data, delimiter=',')

            except Exception as e:
                print(f"Error processing HR data in {folder_name}: {e}")

In [95]:
process_hr_data(processed_dataset_path)

Processing HR Data: 100%|██████████| 13/13 [00:01<00:00,  9.27it/s]


#### Dataframe for All Data

In [96]:
def dataframe_one_folder(serial_number, base_path):
    
    folder_name = f"{serial_number}_unzipped"
    current_folder_path = os.path.join(base_path, folder_name)
    
    # Check if the folder exists
    if os.path.exists(current_folder_path) and os.path.isdir(current_folder_path):
        # Define file paths for EDA, HR_new, TEMP, and STRESS data
        eda_path = os.path.join(current_folder_path, "EDA.csv")
        hr_path = os.path.join(current_folder_path, "HR_new.csv")
        temp_path = os.path.join(current_folder_path, "TEMP.csv")
        stress_path = os.path.join(current_folder_path, f"SM_Drv{serial_number}.csv")

        # Read data frames
        eda = pd.read_csv(eda_path, header=2, names=['EDA'])
        hr = pd.read_csv(hr_path, header=12, names=['HR'])
        temp = pd.read_csv(temp_path, header=2, names=['TEMP'])
        stress = pd.read_csv(stress_path, header=1, names=['STRESS'])
        
        stress = stress * 10

        # Determine the minimum length among data frames
        min_len = min(len(eda), len(hr), len(temp), len(stress))

        # Take the first min_len rows from each data frame
        eda = eda.iloc[:min_len, 0]
        hr = hr.iloc[:min_len, 0]
        temp = temp.iloc[:min_len, 0]
        stress = stress.iloc[:min_len, 0]
        
        # Concatenate the data column-wise
        df_original = pd.concat([eda, hr, temp, stress], axis = 1)
    
        return df_original

In [97]:
def combined_csv(base_path):

    # Initialize an empty DataFrame
    df_combined = pd.DataFrame()

    # Loop through folders with names in the format X_unzipped
    for serial_number in tqdm(range(1, 14), desc="Processing Folders"):
        
        df_original = dataframe_one_folder(serial_number, base_path)
        df_combined = pd.concat([df_combined, df_original], ignore_index=True)

    df_combined.to_csv(f'{base_path}df_combined_data.csv', index=False)
    
    print("CSV File saved successfully: ", df_combined.shape)


In [98]:
combined_csv(processed_dataset_path)

Processing Folders: 100%|██████████| 13/13 [00:01<00:00,  9.52it/s]


CSV File saved successfully:  (194555, 4)


# Pre-processing

### Feature Extraction

In [99]:
def statistical_features(arr):
    vmin = np.amin(arr)
    vmax = np.amax(arr)
    mean = np.mean(arr)
    std = np.std(arr)
    return vmin, vmax, mean, std

def shape_features(arr):
    skewness = skew(arr)
    kurt = kurtosis(arr)
    return skewness, kurt

def calculate_rms(signal):
    diff_squared = np.square(np.ediff1d(signal))
    rms_value = np.sqrt(np.mean(diff_squared))
    return rms_value

### Extended Features

- iterating with a step size of 20
- taking 40 rows at a time to generate a single row of df_features
- find_peaks() : identify peaks in the EDA signal (eda) using the function and then count the number of detected peaks using len() function

In [100]:
def extract_features(data):
    cols = [
        'EDA_Mean', 'EDA_Min', 'EDA_Max', 'EDA_Std', 'EDA_Kurtosis', 'EDA_Skew', 'EDA_Num_Peaks', 'EDA_Amplitude', 'EDA_Duration',
        'HR_Mean', 'HR_Min', 'HR_Max', 'HR_Std', 'HR_RMS', 'TEMP_Mean', 'TEMP_Min', 'TEMP_Max', 'TEMP_Std', 'STRESS'
    ]

    df_features = pd.DataFrame(columns=cols)
    index = 0

    for i in tqdm(range(0, len(data['EDA']), 20), desc="Processing rows", leave=True):
        df_partial = data.iloc[i:i+40,]
        plen = len(df_partial['EDA'])

        if plen < 40:
            continue

        eda = df_partial['EDA'].values
        hr = df_partial['HR'].values
        temp = df_partial['TEMP'].values
        stress = df_partial['STRESS'].values

        eda_min, eda_max, eda_mean, eda_std = statistical_features(eda)
        hr_min, hr_max, hr_mean, hr_std = statistical_features(hr)
        temp_min, temp_max, temp_mean, temp_std = statistical_features(temp)
        stress_min, stress_max, stress_mean, stress_std = statistical_features(stress)
        eda_skew, eda_kurtosis = shape_features(eda)

        hr_rms = calculate_rms(hr)
        temp_rms = calculate_rms(temp)

        peaks, properties = find_peaks(eda, width=5)
        num_Peaks = len(peaks)

        prominences = np.array(properties['prominences'])
        widths = np.array(properties['widths'])
        amplitude = np.sum(prominences)
        duration = np.sum(widths)

        df_features.loc[index] = [eda_mean, eda_min, eda_max, eda_std, eda_kurtosis, eda_skew, num_Peaks, amplitude,
                                  duration, hr_mean, hr_min, hr_max, hr_std, hr_rms, temp_mean, temp_min, temp_max, temp_std, stress_mean]

        index = index + 1

    return df_features

In [101]:
df_combined = pd.read_csv(f'{processed_dataset_path}df_combined_data.csv')

In [102]:
df_features = extract_features(df_combined)

Processing rows: 100%|██████████| 9728/9728 [00:31<00:00, 307.24it/s]


In [103]:
print(df_features.shape)

(9726, 19)


### Lag Features

In [104]:
def generate_lag_features(input_df, columns, lags):
    cols = list(map(str, range(len(columns) * len(lags), 0, -1)))
    lag_df = pd.DataFrame(columns=cols)

    index = len(columns) * len(lags)

    for col in tqdm(columns, desc="Generating lag features", leave=True):
        for lag in tqdm(lags, desc=f"Lag features for {col}", leave=True):
            lagged_column = f'{index}'
            lag_df[lagged_column] = input_df[col].shift(lag)
            index -= 1
    
    lag_df = lag_df.fillna(0)
    
    return lag_df

In [105]:
cols = ['HR_Mean', 'TEMP_Mean', 'EDA_Mean']
lags = [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]

df_lag_features = generate_lag_features(df_features, cols, lags)

Generating lag features: 100%|██████████| 3/3 [00:00<00:00, 80.68it/s]


In [106]:
print(df_lag_features.shape)

(9726, 30)


In [107]:
df_total = pd.concat([df_lag_features, df_features], axis=1)

In [108]:
print(df_total.shape)

(9726, 49)


### Feature and Label Scaling

In [109]:
def scale_label_classifier(df_total, label_column):
    df_total[label_column] = df_total[label_column].apply(lambda x: 0 if x <= 0.325 else (1 if 0.325 < x <= 0.65 else 2))
    return df_total

In [110]:
df_total_scaled_r = df_total.copy()

In [111]:
df_total_scaled_c = df_total.copy()
df_total_scaled_c = scale_label_classifier(df_total_scaled_c, 'STRESS')

Final CSV File for Training

In [112]:
df_total_scaled_r.to_csv(f'{processed_dataset_path}scaled_df_total_regression.csv', index=False)
df_total_scaled_c.to_csv(f'{processed_dataset_path}scaled_df_total_classifier.csv', index=False)

In [113]:
print(df_total_scaled_r.shape, df_total_scaled_c.shape)

(9726, 49) (9726, 49)


# Stress detection model

### Regression Model

In [114]:
data = pd.read_csv(f'{processed_dataset_path}scaled_df_total_regression.csv')

X_r = data.iloc[:,0:48] # features
Y_r = data.iloc[:,48:49] # labels

X_train_r, X_val_r, Y_train_r, Y_val_r = train_test_split(X_r, Y_r, test_size=0.33, random_state=30)

In [115]:
regressor = RandomForestRegressor(n_estimators=100, max_depth=15)
regressor.fit(X_train_r, Y_train_r.values.ravel())

RandomForestRegressor(max_depth=15)

### Classification Model

In [116]:
data = pd.read_csv(f'{processed_dataset_path}scaled_df_total_classifier.csv')

X_c = data.iloc[:,0:48] # features
Y_c = data.iloc[:,48:49] # labels

X_train_c, X_val_c, Y_train_c, Y_val_c = train_test_split(X_c, Y_c, test_size=0.33, random_state=30)

In [117]:
clf = RandomForestClassifier(n_estimators=100, max_depth=15)
clf.fit(X_train_c, Y_train_c.values.ravel())

RandomForestClassifier(max_depth=15)

### Saving Trained Models

In [118]:
date_format = '%Y_%m_%d'  # Format for extracting only the date
current_date_time_dt = dt.datetime.now()  # Current Date and Time in a DateTime Object.
current_date_string = dt.datetime.strftime(current_date_time_dt, date_format) 

In [119]:
# naming system for model
model_file_name_r = f'Regressor_Date_Time_{current_date_string}.pkl'
model_save_path_r = model_path + model_file_name_r

# saving the model
with open(model_save_path_r, 'wb') as file:
    pickle.dump(regressor, file)

In [120]:
# naming system for model
model_file_name_c = f'Classifier_Date_Time_{current_date_string}.pkl'
model_save_path_c = model_path + model_file_name_c

# saving the model
with open(model_save_path_c, 'wb') as file:
    pickle.dump(clf, file)

# Model Analysis

### Regression Model

In [121]:
Y_pred_r = regressor.predict(X_val_r)

In [122]:
score = regressor.score(X_val_r, Y_val_r)
print(f'R-squared Score: {score}')

R-squared Score: 0.8446327405328355


### Classifier Model

In [123]:
Y_pred_c = clf.predict(X_val_c)

In [124]:
f1score   = f1_score        (Y_val_c, Y_pred_c, average = 'macro')
recall    = recall_score    (Y_val_c, Y_pred_c, average = 'macro')
precision = precision_score (Y_val_c, Y_pred_c, average = 'macro')
accuracy  = accuracy_score  (Y_val_c, Y_pred_c)

print('acc =', accuracy)
print('pre =', precision)
print('recall =', recall) 
print('f1 =', f1score)

acc = 0.9834890965732087
pre = 0.654540038970715
recall = 0.6386261243665265
f1 = 0.6462455244107094


c:\Users\rlrnj\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
